# Notebook : Évaluation du modèle sur différentes périodes
charge un modèle de machine learning sauvegardé dans MLflow, évalue les performances du modèle sur deux périodes distinctes. Les résultats sont ensuite enregistrés dans MLflow.



## Importation des bibliothèques nécessaires et configuration mlflow


In [1]:
import os
import joblib
import mlflow
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


In [2]:
# Configure MLflow
mlflow.set_tracking_uri("https://mlflowp51-975919512217.us-central1.run.app")
mlflow.set_experiment("Text_Processing_Experiment")


<Experiment: artifact_location='gs://apip5bucket/artifacts/1', creation_time=1728821200655, experiment_id='1', last_update_time=1728821200655, lifecycle_stage='active', name='Text_Processing_Experiment', tags={}>

## Artefacts
Charge les artefacts stockés dans MLflow (Modèles et données).


In [3]:
# Charge les artefacts (modèles et données)
def load_mlflow_artifact(artifact_path):
    local_path = mlflow.artifacts.download_artifacts(artifact_path)
    with open(local_path, 'rb') as f:
        return joblib.load(f)


In [4]:
# Modèles et les artefacts
X_reduced = load_mlflow_artifact("mlflow_artifacts/X_reduced.pkl")
y = load_mlflow_artifact("mlflow_artifacts/y.pkl")
model_path = "mlflow_artifacts/bow_svd_model.h5"

# Modèle depuis MLflow
model = tf.keras.models.load_model(model_path)

# Compile le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Séparation des données en deux périodes (premiers 6 mois et derniers 6 mois)
Divise les données en deux périodes : janvier à juin 2023 (première période) et juillet à décembre 2023 (deuxième période).


In [5]:
#  Données pour la période de surveillance
df = pd.read_csv('db/cleaned_data_sample.csv')
df['CreationDate'] = pd.to_datetime(df['CreationDate'])

# Divise les données en périodes
df_first_half = df[(df['CreationDate'] >= '2023-01-01') & (df['CreationDate'] < '2023-07-01')]
df_second_half = df[(df['CreationDate'] >= '2023-07-01') & (df['CreationDate'] <= '2023-12-31')]

# Sépare X et y pour chaque période
X_first_half = X_reduced[:len(df_first_half)]
X_second_half = X_reduced[len(df_first_half):]
y_first_half = y[:len(df_first_half)]
y_second_half = y[len(df_first_half):]


In [6]:
def log_descriptive_statistics(df, period_name):
    """Calcule et enregistre les statistiques descriptives des colonnes numériques, avec des noms valides pour MLflow."""
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    descriptive_stats = df[numeric_columns].describe()

    # Normaliser les noms pour MLflow (enlever les caractères non autorisés)
    valid_stats = descriptive_stats.rename(index=lambda x: x.replace('%', 'percent').replace(' ', '_'))

    # Enregistrer les statistiques dans MLflow
    with mlflow.start_run(run_name=f"Descriptive Statistics {period_name}"):
        for col in valid_stats.columns:
            for stat in valid_stats.index:
                metric_name = f"{period_name}_{col}_{stat}"
                mlflow.log_metric(metric_name, valid_stats.loc[stat, col])

    print(f"Statistiques descriptives pour {period_name} enregistrées dans MLflow.")

# Calculer et enregistrer les statistiques descriptives pour les deux périodes
log_descriptive_statistics(df_first_half, "first_half_2023")
log_descriptive_statistics(df_second_half, "second_half_2023")


2024/10/13 14:22:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run Descriptive Statistics first_half_2023 at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1/runs/3c9e656644754c288b365b64a556721e.
2024/10/13 14:22:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1.


RestException: INVALID_PARAMETER_VALUE: Invalid metric name: 'first_half_2023_Id_25%'. Names may only contain alphanumerics, underscores (_), dashes (-), periods (.), spaces ( ), and slashes (/).

## Évaluation du modèle sans réentraînement

Cette fonction évalue les performances du modèle sans le réentraîner sur les deux périodes (data drift et model drift).


In [6]:
# Évaluation du modèle sans réentraînement pour chaque période
def evaluate_model(model, X_test, y_test, dataset_name):
    """Évalue les performances du modèle, y compris la perte, sans le réentraîner."""
    # Prédictions du modèle
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)  # Ajout de la perte lors de l'évaluation
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype("int32")
    
    # Calcul des métriques
    f1 = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')

    # Enregistrement des résultats dans MLflow
    with mlflow.start_run(run_name=f"Model Evaluation {dataset_name}"):
        mlflow.log_metric(f"{dataset_name}_loss", loss)
        mlflow.log_metric(f"{dataset_name}_accuracy", accuracy)
        mlflow.log_metric(f"{dataset_name}_f1_score", f1)
        mlflow.log_metric(f"{dataset_name}_precision", precision)
        mlflow.log_metric(f"{dataset_name}_recall", recall)

    print(f"{dataset_name} - Loss: {loss}, Accuracy: {accuracy}, F1 Score: {f1}")


## Évaluation du modèle sur les deux semestres d'un an (Data Drift et Model Drift)
Compare les performances du modèle sur les deux périodes pour détecter les dérives de données et de modèle.


In [7]:
# Performances pour chaque période sans réentraînement
evaluate_model(model, X_first_half, y_first_half, "first_half_2023")
evaluate_model(model, X_second_half, y_second_half, "second_half_2023")

# Affichage des résultats
print("Évaluation du modèle enregistrée dans MLflow.")


499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step


2024/10/13 14:06:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model Evaluation first_half_2023 at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1/runs/7f9e997515a843378ff0cbe9f8e49ccc.
2024/10/13 14:06:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1.


first_half_2023 - Accuracy: 0.38625963768570176, F1 Score: 0.46978230124867953
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step


c:\Users\leenc\Documents\openclassrooms\p5\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2024/10/13 14:06:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run Model Evaluation second_half_2023 at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1/runs/8d48cc8ea246474db8030ea10020c9f1.
2024/10/13 14:06:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflowp51-975919512217.us-central1.run.app/#/experiments/1.


second_half_2023 - Accuracy: 0.3811188811188811, F1 Score: 0.45644860539824234
Évaluation du modèle enregistrée dans MLflow.
